In [10]:
import zipfile
import boto3
from botocore.client import Config
import os
import json

In [7]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

In [8]:
with zipfile.ZipFile('replacement.zip', 'r') as zipf:
    zipf.extractall()

In [11]:
with open('should_replace.json', 'r') as f:
    decision = json.load(f)

In [ ]:
project_name_param = os.getenv('project_name')

In [40]:
def empty_production_folder():
    objects_to_delete = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=f"{project_name_param}/production/")

    if 'Contents' in objects_to_delete:
        delete_keys = {'Objects': []}
        for obj in objects_to_delete['Contents']:
            delete_keys['Objects'].append({'Key': obj['Key']})
        s3_client.delete_objects(Bucket=bucket_name, Delete=delete_keys)
    else:
        print("No objects found")

In [41]:
def find_model_file():
    for file in os.listdir(os.getcwd()):
        if file.endswith('.onnx'):
            return file

    return None

In [42]:
model = find_model_file()
folder_name = f"{project_name_param}/production/"

def upload_to_production_folder():
    s3_client.upload_file(model, bucket_name, folder_name + model)

In [43]:
if decision['replace']:
    empty_production_folder()
    upload_to_production_folder()
else:
    print("Current model retained in production.")